In [1]:
#Loading of dataset IMDB
!pip install datasets
from datasets import load_dataset

dataset = load_dataset("imdb")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 5.0 MB/s 
     |████████████████████████████████| 132 kB 19.5 MB/s 
     |████████████████████████████████| 182 kB 53.8 MB/s 
     |████████████████████████████████| 212 kB 52.6 MB/s 
     |████████████████████████████████| 127 kB 42.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
dataset["train"][100]

{'text': "Terrible movie. Nuff Said.<br /><br />These Lines are Just Filler. The movie was bad. Why I have to expand on that I don't know. This is already a waste of my time. I just wanted to warn others. Avoid this movie. The acting sucks and the writing is just moronic. Bad in every way. The only nice thing about the movie are Deniz Akkaya's breasts. Even that was ruined though by a terrible and unneeded rape scene. The movie is a poorly contrived and totally unbelievable piece of garbage.<br /><br />OK now I am just going to rag on IMDb for this stupid rule of 10 lines of text minimum. First I waste my time watching this offal. Then feeling compelled to warn others I create an account with IMDb only to discover that I have to write a friggen essay on the film just to express how bad I think it is. Totally unnecessary.",
 'label': 0}

In [3]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.1 MB/s 
     |████████████████████████████████| 7.6 MB 38.5 MB/s 


In [4]:
# from transformers import BertTokenizer, TFBertForSequenceClassification
# from transformers import InputExample, InputFeatures

# model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [5]:
# Using Pretrained Auto Tokenizer
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("fabriceyhc/bert-base-uncased-imdb")


def tokenize_function(examples):
    return tokenizer(examples["text"], max_length=512, truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Downloading:   0%|          | 0.00/321 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

In [6]:
# Train Test Split
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(100))

In [7]:
small_eval_dataset.shape

(100, 5)

In [8]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("fabriceyhc/bert-base-uncased-imdb", num_labels=2)

Downloading:   0%|          | 0.00/826 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [10]:
# evaulating the model on test data
from transformers import pipeline
model_name = "fabriceyhc/bert-base-uncased-imdb"
nlp = pipeline("sentiment-analysis", model=model_name, tokenizer=model_name)
tp =0
for  i in  range(len(small_eval_dataset)):
       text = small_eval_dataset[i]["text"]
       if len(text) > 512:
             text = text[:512]
       results = nlp(text)
       print(results)
       if results[0]['label']=='pos':
          results[0]['label']=1
       if results[0]['label']=='neg':
          results[0]['label']=0
           
       if results[0]['label']==small_eval_dataset[i]['label']:
          tp=tp+1
       
print(tp/(len(small_eval_dataset)))              

[{'label': 'pos', 'score': 0.9996671676635742}]
[{'label': 'pos', 'score': 0.9996638298034668}]
[{'label': 'neg', 'score': 0.9924436211585999}]
[{'label': 'pos', 'score': 0.9996439218521118}]
[{'label': 'neg', 'score': 0.9996143579483032}]
[{'label': 'pos', 'score': 0.9996660947799683}]
[{'label': 'pos', 'score': 0.9996684789657593}]
[{'label': 'neg', 'score': 0.9996353387832642}]
[{'label': 'neg', 'score': 0.9995144605636597}]
[{'label': 'pos', 'score': 0.9996663331985474}]
[{'label': 'pos', 'score': 0.9996657371520996}]
[{'label': 'neg', 'score': 0.9996557235717773}]
[{'label': 'pos', 'score': 0.9924954175949097}]
[{'label': 'neg', 'score': 0.9961469173431396}]
[{'label': 'pos', 'score': 0.9996672868728638}]
[{'label': 'pos', 'score': 0.9996678829193115}]
[{'label': 'pos', 'score': 0.9955743551254272}]
[{'label': 'pos', 'score': 0.9931942224502563}]
[{'label': 'pos', 'score': 0.9996693134307861}]
[{'label': 'pos', 'score': 0.9996651411056519}]
[{'label': 'neg', 'score': 0.99958723783